In [27]:
import time
import os
import re

import pandas as pd

province_dict = {
    "Anhui": "安徽省",
    "Beijing": "北京市",
    "Chongqin": "重庆市",
    "Chongqing": "重庆市",
    "Fujian": "福建省",
    "Gansu": "甘肃省",
    "Guangdong": "广东省",
    "Guangxi": "广西壮族自治区",
    "Guizhou": "贵州省",
    "Neimenggu": "内蒙古自治区",
    "Hainan": "海南省",
    "Hebei": "河北省",
    "Henan": "河南省",
    "Heilongjiang": "黑龙江省",
    "HeiLongJiang": "黑龙江省",
    "Hubei": "湖北省",
    "Hunan": "湖南省",
    "Jiangsu": "江苏省",
    "Jilin": "吉林省",
    "Jiangxi": "江西省",
    "Liaoning": "辽宁省",
    "NeiMenggu": "内蒙古自治区",
    "Ningxia": "宁夏回族自治区",
    "Qinghai": "青海省",
    "Shandong": "山东省",
    "Shanghai": "上海市",
    "Shannxi": "陕西省",
    "Shanxi": "山西省",
    "Sichuan": "四川省",
    "Tianjing": "天津市",
    "Tianjin": "天津市",
    "Xinjiang": "新疆维吾尔自治区",
    "Xizang": "西藏自治区",
    "Yunnan": "云南省",
    "Zhejiang": "浙江省"
}

# 省份编码表
province_codes = {
    '北京市': 1, '天津市': 2, '上海市': 3, '重庆市': 4, '河北省': 5, '福建省': 6, '江西省': 7, '山东省': 8,
    '山西省': 9, '内蒙古自治区': 10, '河南省': 11, '辽宁省': 12, '湖北省': 13, '吉林省': 14, '湖南省': 15,
    '黑龙江省': 16, '广东省': 17, '江苏省': 18, '广西壮族自治区': 19, '海南省': 20, '四川省': 21, '浙江省': 22,
    '贵州省': 23, '安徽省': 24, '云南省': 25, '陕西省': 26, '西藏藏族自治区': 27, '甘肃省': 28, '青海省': 29,
    '宁夏回族自治区': 30, '新疆维吾尔自治区': 31
}

# 城市编码表
city_codes = {
    "北京市": 1,
    "天津市": 2,
    "石家庄市": 3,
    "唐山市": 4,
    "秦皇岛市": 5,
    "邯郸市": 6,
    "邢台市": 7,
    "保定市": 8,
    "张家口市": 9,
    "承德市": 10,
    "沧州市": 11,
    "廊坊市": 12,
    "衡水市": 13,
    "太原市": 14,
    "大同市": 15,
    "阳泉市": 16,
    "长治市": 17,
    "晋城市": 18,
    "朔州市": 19,
    "晋中市": 20,
    "运城市": 21,
    "忻州市": 22,
    "临汾市": 23,
    "吕梁市": 24,
    "呼和浩特市": 25,
    "包头市": 26,
    "乌海市": 27,
    "赤峰市": 28,
    "通辽市": 29,
    "鄂尔多斯市": 30,
    "呼伦贝尔市": 31,
    "巴彦淖尔市": 32,
    "乌兰察布市": 33,
    "沈阳市": 34,
    "大连市": 35,
    "鞍山市": 36,
    "抚顺市": 37,
    "本溪市": 38,
    "丹东市": 39,
    "锦州市": 40,
    "营口市": 41,
    "阜新市": 42,
    "辽阳市": 43,
    "盘锦市": 44,
    "铁岭市": 45,
    "朝阳市": 46,
    "葫芦岛市": 47,
    "长春市": 48,
    "吉林市": 49,
    "四平市": 50,
    "辽源市": 51,
    "通化市": 52,
    "白山市": 53,
    "松原市": 54,
    "白城市": 55,
    "哈尔滨市": 56,
    "齐齐哈尔市": 57,
    "鸡西市": 58,
    "鹤岗市": 59,
    "双鸭山市": 60,
    "大庆市": 61,
    "伊春市": 62,
    "佳木斯市": 63,
    "七台河市": 64,
    "牡丹江市": 65,
    "黑河市": 66,
    "绥化市": 67,
    "上海市": 68,
    "南京市": 69,
    "无锡市": 70,
    "徐州市": 71,
    "常州市": 72,
    "苏州市": 73,
    "南通市": 74,
    "连云港市": 75,
    "淮安市": 76,
    "盐城市": 77,
    "扬州市": 78,
    "镇江市": 79,
    "泰州市": 80,
    "宿迁市": 81,
    "杭州市": 82,
    "宁波市": 83,
    "温州市": 84,
    "嘉兴市": 85,
    "湖州市": 86,
    "绍兴市": 87,
    "金华市": 88,
    "衢州市": 89,
    "舟山市": 90,
    "台州市": 91,
    "丽水市": 92,
    "合肥市": 93,
    "芜湖市": 94,
    "蚌埠市": 95,
    "淮南市": 96,
    "马鞍山市": 97,
    "淮北市": 98,
    "铜陵市": 99,
    "安庆市": 100,
    "黄山市": 101,
    "滁州市": 102,
    "阜阳市": 103,
    "宿州市": 104,
    "六安市": 105,
    "亳州市": 106,
    "池州市": 107,
    "宣城市": 108,
    "福州市": 109,
    "厦门市": 110,
    "莆田市": 111,
    "三明市": 112,
    "泉州市": 113,
    "漳州市": 114,
    "南平市": 115,
    "龙岩市": 116,
    "宁德市": 117,
    "南昌市": 118,
    "景德镇市": 119,
    "萍乡市": 120,
    "九江市": 121,
    "新余市": 122,
    "鹰潭市": 123,
    "赣州市": 124,
    "吉安市": 125,
    "宜春市": 126,
    "抚州市": 127,
    "上饶市": 128,
    "济南市": 129,
    "青岛市": 130,
    "淄博市": 131,
    "枣庄市": 132,
    "东营市": 133,
    "烟台市": 134,
    "潍坊市": 135,
    "济宁市": 136,
    "泰安市": 137,
    "威海市": 138,
    "日照市": 139,
    "临沂市": 140,
    "德州市": 141,
    "聊城市": 142,
    "滨州市": 143,
    "菏泽市": 144,
    "郑州市": 145,
    "开封市": 146,
    "洛阳市": 147,
    "平顶山市": 148,
    "安阳市": 149,
    "鹤壁市": 150,
    "新乡市": 151,
    "焦作市": 152,
    "濮阳市": 153,
    "许昌市": 154,
    "漯河市": 155,
    "三门峡市": 156,
    "南阳市": 157,
    "商丘市": 158,
    "信阳市": 159,
    "周口市": 160,
    "驻马店市": 161,
    "武汉市": 162,
    "黄石市": 163,
    "十堰市": 164,
    "宜昌市": 165,
    "襄阳市": 166,
    "鄂州市": 167,
    "荆门市": 168,
    "孝感市": 169,
    "荆州市": 170,
    "黄冈市": 171,
    "咸宁市": 172,
    "随州市": 173,
    "长沙市": 174,
    "株洲市": 175,
    "湘潭市": 176,
    "衡阳市": 177,
    "邵阳市": 178,
    "岳阳市": 179,
    "常德市": 180,
    "张家界市": 181,
    "益阳市": 182,
    "郴州市": 183,
    "永州市": 184,
    "怀化市": 185,
    "娄底市": 186,
    "广州市": 187,
    "韶关市": 188,
    "深圳市": 189,
    "珠海市": 190,
    "汕头市": 191,
    "佛山市": 192,
    "江门市": 193,
    "湛江市": 194,
    "茂名市": 195,
    "肇庆市": 196,
    "惠州市": 197,
    "梅州市": 198,
    "汕尾市": 199,
    "河源市": 200,
    "阳江市": 201,
    "清远市": 202,
    "东莞市": 203,
    "中山市": 204,
    "潮州市": 205,
    "揭阳市": 206,
    "云浮市": 207,
    "南宁市": 208,
    "柳州市": 209,
    "桂林市": 210,
    "梧州市": 211,
    "北海市": 212,
    "防城港市": 213,
    "钦州市": 214,
    "贵港市": 215,
    "玉林市": 216,
    "百色市": 217,
    "贺州市": 218,
    "河池市": 219,
    "来宾市": 220,
    "崇左市": 221,
    "海口市": 222,
    "三亚市": 223,
    "三沙市": 224,
    "儋州市": 225,
    "重庆市": 226,
    "成都市": 227,
    "自贡市": 228,
    "攀枝花市": 229,
    "泸州市": 230,
    "德阳市": 231,
    "绵阳市": 232,
    "广元市": 233,
    "遂宁市": 234,
    "内江市": 235,
    "乐山市": 236,
    "南充市": 237,
    "眉山市": 238,
    "宜宾市": 239,
    "广安市": 240,
    "达州市": 241,
    "雅安市": 242,
    "巴中市": 243,
    "资阳市": 244,
    "贵阳市": 245,
    "六盘水市": 246,
    "遵义市": 247,
    "安顺市": 248,
    "毕节市": 249,
    "铜仁市": 250,
    "昆明市": 251,
    "曲靖市": 252,
    "玉溪市": 253,
    "保山市": 254,
    "昭通市": 255,
    "丽江市": 256,
    "普洱市": 257,
    "临沧市": 258,
    "拉萨市": 259,
    "日喀则市": 260,
    "昌都市": 261,
    "林芝市": 262,
    "山南市": 263,
    "那曲市": 264,
    "西安市": 265,
    "铜川市": 266,
    "宝鸡市": 267,
    "咸阳市": 268,
    "渭南市": 269,
    "延安市": 270,
    "汉中市": 271,
    "榆林市": 272,
    "安康市": 273,
    "商洛市": 274,
    "兰州市": 275,
    "嘉峪关市": 276,
    "金昌市": 277,
    "白银市": 278,
    "天水市": 279,
    "武威市": 280,
    "张掖市": 281,
    "平凉市": 282,
    "酒泉市": 283,
    "庆阳市": 284,
    "定西市": 285,
    "陇南市": 286,
    "西宁市": 287,
    "海东市": 288,
    "银川市": 289,
    "石嘴山市": 290,
    "吴忠市": 291,
    "固原市": 292,
    "中卫市": 293,
    "乌鲁木齐市": 294,
    "克拉玛依市": 295,
    "吐鲁番市": 296,
    "哈密市": 297
}

city_name_map = {
    'Kunming': '昆明市',
    'Puer': '普洱市',
    'Baliaonier': '巴彦淖尔市',
    'Baotou': '包头市',
    'Wuhai': '乌海市',
    'Baicheng': '白城市',
    'Changchun': '长春市',
    'Jilin': '吉林市',
    'Liaoyuan': '辽源市',
    'Sipin': '四平市',
    'Songyuan': '松原市',
    'Tonghua': '通化市',
    'Bazhong': '巴中市',
    'Guangan': '广安市',
    'Mianyang': '绵阳市',
    'Nanjing': '南京市',
    'Panzihua': '攀枝花市',
    'Yaan': '雅安市',
    'Yibing': '宜宾市',
    'Yueshan': '乐山市',
    'Zigong': '自贡市',
    'Zhongwei': '中卫市',
    'Anqing': '安庆市',
    'Bengbu': '蚌埠市',
    'Chizhou': '池州市',
    'Fuyan': '阜阳市',
    'Haozhou': '毫州市',  # 需核实地名，可能是淮北市（Huaibei）
    'Hefei': '合肥市',
    'Huaibei': '淮北市',
    'Huainan': '淮南市',
    'Huangshan': '黄山市',
    'liuan': '六安市',
    'Maanshan': '马鞍山市',
    'Suzhou': '宿州市',
    'Tongling': '铜陵市',
    'Wuhu': '芜湖市',
    'Xuancheng': '宣城市',
    'Binzhou': '滨州市',
    'Hezhe': '菏泽市',  # 需核实地名，可能是某个少数民族自治县级单位
    'Jinan': '济南市',
    'Jining': '济宁市',
    'Liaocheng': '聊城市',
    'Linyi': '临沂市',
    'Qingdao': '青岛市',
    'Rizhao': '日照市',
    'Taian': '泰安市',
    'Weihai': '威海市',
    'Yantai': '烟台市',
    'Zibo': '淄博市',
    'Jincheng': '晋城市',
    'Jinzhong': '晋中市',
    'Linfen': '临汾市',
    'Lvliang': '吕梁市',
    'Shuozhou': '朔州市',
    'Chaozhou': '潮州市',
    'Dongguan': '东莞市',
    'Foshan': '佛山市',
    'Guangzhou': '广州市',
    'Heyuan': '河源市',
    'Huizhou': '惠州市',
    'Jiangmen': '江门市',
    'Jieyang': '揭阳市',
    'Maoming': '茂名市',
    'Meizhou': '梅州市',
    'Qingyuan': '清远市',
    'Shantou': '汕头市',
    'Shanwei': '汕尾市',
    'Shaoguan': '韶关市',
    'Shenzhen': '深圳市',
    'Yangjiang': '阳江市',
    'Yunfu': '云浮市',
    'Zhanjiang': '湛江市',
    'Zhaoqing': '肇庆市',
    'Zhongshan': '中山市',
    'Zhuhai': '珠海市',
    'Baise': '百色市',
    'Chongzuo': '崇左市',
    'Fangchenggan': '防城港市',
    'Guigan': '贵港市',
    'Hechi': '河池市',
    'Hezhou': '贺州市',
    'Laibin': '来宾市',
    'Liuzhou': '柳州市',
    'Nanning': '南宁市',
    'Yulin': '玉林市',
    'Changzhou': '常州市',
    'Nantong': '南通市',
    'Wuxi': '无锡市',
    'Yancheng': '盐城市',
    'Ganzhou': '赣州市',
    'Jian': '吉安市',
    'Nanchang': '南昌市',
    'Pingxiang': '萍乡市',
    'Shangrao': '上饶市',
    'Xinyu': '新余市',
    'Yichun': '宜春市',
    'Yingtan': '鹰潭市',
    'Cangzhou': '沧州市',
    'Chengde': '承德市',
    'Xingtai': '邢台市',
    '秦皇岛市': '秦皇岛市',
    'Anyang': '安阳市',
    'Jiaozuo': '焦作市',
    'Kaifeng': '开封市',
    'Luohe': '漯河市',
    'Luoyang': '洛阳市',
    'Pingdingshan': '平顶山市',
    'Shangqiu': '商丘市',
    'Xinxiang': '新乡市',
    'Xuchang': '许昌市',
    'Zhengzhou': '郑州市',
    'Zhoukou': '周口市',
    'Zhumadian': '驻马店市',
    'Hangzhou': '杭州市',
    'Huzhou': '湖州市',
    'Jiaxin': '嘉兴市',
    'Jinhua': '金华市',
    'Lishui': '丽水市',
    'Ningbo': '宁波市',
    'Quzhou': '衢州市',
    'Taizhou': '台州市',
    'Wenzhou': '温州市',
    'Zhaoxin': '绍兴市', 
    'Zhoushan': '舟山市',
    'Huanggang': '黄冈市',
    'Jingzhou': '荆州市',
    'Suizhou': '随州市',
    'Xianning': '咸宁市',
    'Xiaogan': '孝感市',
    'Yichang': '宜昌市',
    'Changsha': '长沙市',
    'Xiangtan': '湘潭市',
    'Yiyang': '益阳市',
    'Baiyin': '白银市',
    'Jiayuguan': '嘉峪关市',
    'Jiuquan': '酒泉市',
    'Lanzhou': '兰州市',
    'Pingliang': '平凉市',
    'Qingyang': '庆阳市',
    'Tianshui': '天水市',
    'Wuwei': '武威市',
    'Fuzhou': '福州市',
    'Guiyang': '贵阳市',
    'Liupanshui': '六盘水市',
    'Tongren': '铜仁市',
    'Zunyi': '遵义市',
    'Chaoyang': '朝阳市',
    'Fuxin': '阜新市',
    'Jinzhou': '锦州市',
    'Yinkou': '营口市',
    'Baoji': '宝鸡市',
    'Xianyang': '咸阳市',
    'Xining': '西宁市',
    'Heihe': '黑河市',
    'Jiamusi': '佳木斯市',
    'Jixi': '鸡西市',
    'Mudanjiang': '牡丹江市',
    'Qiqihaer': '齐齐哈尔市'
}


In [28]:
def print_xlsx_in_directory(directory):
    try:
        # 遍历目录中的所有项目
        for root, dirs, files in os.walk(directory):
            for filename in files:
                if filename.endswith(".xlsx"):
                    print(filename)
    except Exception as e:
        print(f"An error occurred: {e}")

In [29]:

def rename_folders_in_directory(directory, province_dict):
    try:
        # 遍历目录中的所有项目
        for item in os.listdir(directory):
            item_path = os.path.join(directory, item)
            # 检查项目是否为文件夹
            if os.path.isdir(item_path):
                # 获取文件夹的拼音名称
                pinyin_name = item
                # 根据拼音名称查找对应的官方名称
                if pinyin_name in province_dict:
                    official_name = province_dict[pinyin_name]
                    new_item_path = os.path.join(directory, official_name)
                    # 重命名文件夹
                    os.rename(item_path, new_item_path)
                    print(f"Renamed '{item}' to '{official_name}'")
                else:
                    print(f"No mapping found for folder '{item}'")
    except Exception as e:
        print(f"An error occurred: {e}")
        
def rename_files_in_directory(directory, city_name_map):
    try:
        # 遍历目录中的所有项目
        for root, dirs, files in os.walk(directory):
            # 获取当前文件夹的名称
            province_name = os.path.basename(root)
            
            for filename in files:
                # 检查项目是否为xlsx文件
                if filename.endswith('news.xlsx'):
                    # 获取文件夹的拼音名称
                    pinyin_name = filename.split('_')[0]
                    # 根据拼音名称查找对应的官方名称
                    if pinyin_name in city_name_map:
                        official_name = city_name_map[pinyin_name]
                        
                        # 读取 Excel 文件
                        df = pd.read_excel(os.path.join(root, filename))
                        
                        required_columns = ['topic', 'date', 'url', 'content']
                        # 检查必要的列是否存在
                        if not all(col in df.columns for col in required_columns):
                            print(f"{filename} - 某些必要列缺失")
                            continue
                        
                        df['city'] = official_name
                        df['province'] = province_name
                        
                        # 添加省份编号列
                        df['code'] = df['city'].map(city_codes, na_action='ignore')
                        
                        if df['code'].isnull().any():
                            print(f"{filename} - 省份编码缺失")
                        
                        df = df[['code', 'province', 'city', 'topic', 'date', 'url', 'content']]
                        
                        os.remove(os.path.join(root, filename))
                        
                        news_path = os.path.join(root, f"{official_name}_学习考察.xlsx")
                        df.to_excel(news_path, index=False)

                        print(f"{news_path} - 增加新列成功")
                    else:
                        print(f"{filename} - 无对应官方名称")
                else:
                    print(f"{filename} - 不是一个有效的文件")
    except Exception as e:
        print(f" An error occurred: {e}")

def mathc_province_code(directory):
    # 检查目录是否存在
    if not os.path.isdir(directory):
        print("提供的路径不是一个有效的目录")
        return

    num_of_all_files = 0
    num_of_targeted_files = 0
    skip_files_list = []

    # 遍历目录及其子目录中的所有 Excel 文件
    for root, dirs, files in os.walk(directory):
        
        for filename in files:
            if filename.endswith(".xlsx"):
                num_of_all_files += 1
            
            if filename.endswith('学习考察.xlsx'):
                file_path = os.path.join(root, filename)
                print(f"获取 {filename} 成功")
                num_of_targeted_files += 1
                # 读取 Excel 文件
                df = pd.read_excel(file_path)
            
                # 检查必要的列是否存在
                required_columns = ['province', 'city', 'topic', 'date', 'url', 'content']
                for column in required_columns:
                    if column not in df.columns:
                        print(f"缺少 {column} 列")
                
                # 添加省份编号列
                df['code'] = df['province'].map(province_codes, na_action='ignore')
                
                if df['code'].isnull().any():
                    print(f"{filename}省份编码缺失")
                
                df = df[['code', 'province', 'topic', 'date', 'url', 'content']]
                
                # 保存为原来文件夹的路径，并且文件名相同
                base_name, ext = os.path.splitext(filename)
                new_file_path = os.path.join(root, f"{base_name}{ext}")
                df.to_excel(new_file_path, index=False)
                print(f"保存 {filename} 成功")
                
            elif filename.endswith('.xlsx'): 
                skip_files_list.append(filename)
                
    print(f"共获取 {num_of_targeted_files} 个有效文件，共 {num_of_all_files} 个文件")
    print(f"跳过 {len(skip_files_list)} 个文件：{skip_files_list}")
    
def mathc_city_code(directory):
    # 检查目录是否存在
    if not os.path.isdir(directory):
        print("提供的路径不是一个有效的目录")
        return

    num_of_all_files = 0
    num_of_targeted_files = 0
    skip_files_list = []

    # 遍历目录及其子目录中的所有 Excel 文件
    for root, dirs, files in os.walk(directory):
        
        for filename in files:
            if filename.endswith(".xlsx"):
                num_of_all_files += 1
            
            if filename.endswith('学习考察.xlsx'):
                file_path = os.path.join(root, filename)
                print(f"获取 {filename} 成功")
                num_of_targeted_files += 1
                # 读取 Excel 文件
                df = pd.read_excel(file_path)
            
                # 检查必要的列是否存在
                required_columns = ['province', 'city', 'topic', 'date', 'url', 'content']
                if not all(col in df.columns for col in required_columns):
                    print(f"{filename} - 某些必要列缺失")
                    continue
                
                # 添加省份编号列
                df['code'] = df['city'].map(city_codes, na_action='ignore')
                
                if df['code'].isnull().any():
                    print(f"{filename} - 省份编码缺失")
                
                df = df[['code', 'province', 'city', 'topic', 'date', 'url', 'content']]
                
                # 保存为原来文件夹的路径，并且文件名相同
                base_name, ext = os.path.splitext(filename)
                new_file_path = os.path.join(root, f"{base_name}{ext}")
                df.to_excel(new_file_path, index=False)
                print(f"保存 {filename} 成功")
                
            elif filename.endswith('.xlsx'): 
                skip_files_list.append(filename)
                
    print(f"共获取 {num_of_targeted_files} 个有效文件，共 {num_of_all_files} 个文件")
    print(f"跳过 {len(skip_files_list)} 个文件：{skip_files_list}")

In [30]:
total_cities_path = r'E:\pythonProject\outsource\Crawler_of_China_govern_website\output\Total_time_range_data'

certain_cities_path = r'E:\pythonProject\outsource\Crawler_of_China_govern_website\output\Certain_time_range_data'

province_path = r'E:\pythonProject\outsource\Crawler_of_China_govern_website\output\province_web_data'

# rename_folders_in_directory(certain_cities_path, province_dict)
# mathc_province_code(province_path)
# mathc_city_code(total_cities_path)

# print_xlsx_in_directory(certain_cities_path)
rename_files_in_directory(certain_cities_path, city_name_map)

E:\pythonProject\outsource\Crawler_of_China_govern_website\output\Certain_time_range_data\云南省\昆明市_学习考察.xlsx - 增加新列成功
E:\pythonProject\outsource\Crawler_of_China_govern_website\output\Certain_time_range_data\云南省\普洱市_学习考察.xlsx - 增加新列成功
昆明市_学习考察.xlsx - 不是一个有效的文件
普洱市_学习考察.xlsx - 不是一个有效的文件
E:\pythonProject\outsource\Crawler_of_China_govern_website\output\Certain_time_range_data\内蒙古自治区\巴彦淖尔市_学习考察.xlsx - 增加新列成功
E:\pythonProject\outsource\Crawler_of_China_govern_website\output\Certain_time_range_data\内蒙古自治区\包头市_学习考察.xlsx - 增加新列成功
E:\pythonProject\outsource\Crawler_of_China_govern_website\output\Certain_time_range_data\内蒙古自治区\乌海市_学习考察.xlsx - 增加新列成功
乌海市_学习考察.xlsx - 不是一个有效的文件
包头市_学习考察.xlsx - 不是一个有效的文件
巴彦淖尔市_学习考察.xlsx - 不是一个有效的文件
E:\pythonProject\outsource\Crawler_of_China_govern_website\output\Certain_time_range_data\吉林省\白城市_学习考察.xlsx - 增加新列成功
E:\pythonProject\outsource\Crawler_of_China_govern_website\output\Certain_time_range_data\吉林省\长春市_学习考察.xlsx - 增加新列成功
E:\pythonProject\outsource\Crawler_of_